In [7]:
from pathlib import Path

import pandas as pd

import utils

In [12]:
filename = 'data/watson_healthcare_modified.csv'

df = pd.read_csv(Path(utils.root(), filename), sep=',')
print(df['Age'].mean())

36.866348448687354
